<a href="https://colab.research.google.com/github/yeop6134/Ai_humanities/blob/main/BERT%EA%B8%B0%EB%B0%98TopicModeling/Korean_CTM_%EB%AA%85%EC%82%AC%EC%82%AC%EC%9A%A9_%EB%B6%88%EC%9A%A9%EC%96%B4%EC%A0%9C%EA%B1%B0_Okt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/인공지능인문학"

/content/drive/MyDrive/Colab Notebooks/인공지능인문학


In [ ]:
# with open("discrimination.txt", "r", encoding='utf-8-sig') as f:
#     text_file = f.readlines()

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab, Okt
from tqdm import tqdm

In [ ]:
documents = [line.strip() for line in open("discrimination.txt", encoding="utf-8").readlines()]

In [ ]:
documents[:5]

['text',
 '남자들은 도대체 왜 그러는걸까?',
 '말하면서 침 개많이 튀길것 같아.',
 '머리가 멍청해서 못읽는거아니냐?',
 '"동유럽, 러시아 애들은 가난해도 사랑으로 결혼하는데 헬조선은 자본주의의 지옥 그 자체다"']

In [ ]:
korean_stopwords_path = "./korean_stopwords.txt"

# 텍스트 파일을 오픈합니다.
with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.nouns(sent)
        words = [word for word in word_tokens if len(word) > 1]

        result = []
        for w in words: 
         if w not in stopwords: 
            result.append(w)

        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Okt())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [ ]:
train_bow_embeddings = vectorizer.fit_transform(documents)

In [ ]:
print(train_bow_embeddings.shape)

(4436, 3000)


In [ ]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
len(vocab)

3000

In [ ]:
train_contextualized_embeddings = bert_embeddings_from_list(documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

# Combined TM 학습하기

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=8, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [88720/88720]	Train Loss: 22.285956283858276	Time: 0:00:01.321164: : 20it [00:31,  1.56s/it]


# 토픽들
학습 후에는 토픽 모델이 선정한 토픽들을 보려면 아래의 메소드를 사용합니다.  


> get_topic_lists


해당 메소드에는 각 토픽마다 몇 개의 단어를 보고 싶은지에 해당하는 파라미터를 넣어즐 수 있습니다.

In [ ]:
ctm.get_topics(10)

defaultdict(list,
            {0: ['돼지', '뭔가', '머리', '피하', '정신', '쓰레기', '백인', '뒤통수', '의심', '피부'],
             1: ['조선족',
              '중국',
              '한국',
              '중국인',
              '서울',
              '나라',
              '북한',
              '한국인',
              '세계',
              '이제'],
             2: ['흑인', '차별', '요즘', '사람', '공부', '동남아', '장애', '냄새', '장애인', '인종'],
             3: ['부감', '부들', '미스김', '거울', '이틀', '한잔', '더욕', '기생', '살이', '기회'],
             4: ['여자',
              '남자',
              '결혼',
              '원래',
              '여자애',
              '운전',
              '여성',
              '무조건',
              '자고로',
              '소방관'],
             5: ['아프리카',
              '가라',
              '고급',
              '벅스',
              '백수',
              '싸개',
              '경찰서',
              '집세',
              '넥타이',
              '원숭이'],
             6: ['무조건',
              '얼굴',
              '여자애',
              '운전',
              '게이',
              

# 시각화

In [ ]:
%pip install pyLDAvis

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)
     

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
Sampling: [10/10]: : 10it [00:15,  1.55s/it]
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
